In [1]:
import os
os.chdir("..")
os.getcwd()

'C:\\Users\\gwenk\\OneDrive\\Documents\\Research\\Scraping\\diseaseScraping'

In [2]:
import pandas as pd
import pyarrow
import numpy as np
from bs4 import BeautifulSoup

In [3]:
directory = "C:\\Users\\gwenk\\OneDrive\\Documents\\Research\\Scraping\\diseaseScraping\\Diseases"
file_list = [file for file in os.listdir(directory) if file.endswith('.parquet')]
print(file_list)

['AADA.parquet', 'CanadaPublicHealth.parquet', 'CDC.parquet', 'Cleveland Clinic.parquet', 'Clinical Trials gov.parquet', 'DrugsDOTcom.parquet', 'HealthDirectGovAU.parquet', 'Healthy Western Australians (gov).parquet', 'Hexahealth.parquet', 'IDPH.parquet', 'Mayoclinic.parquet', 'MedlinePlus.parquet', 'Medscape.parquet', 'Merck Manuals.parquet', 'NHS.parquet', 'NHSScot.parquet', 'NIH NINDS.parquet', 'NIH.parquet', 'Rare Diseases NIH.parquet', 'RareDiseases.org.parquet', 'Rheumatology.parquet', 'Seattle Childrens.parquet', 'WebMD.parquet', 'WHO.parquet', 'Wikipedia.parquet', 'Wisconsin Department of Health Services.parquet']


In [4]:
file_list.remove("RareDiseases.org.parquet")

In [5]:
dataframes = []
for file in file_list:
    file_path = os.path.join(directory, file)
    df = pd.read_parquet(file_path,engine='pyarrow')
    dataframes.append(df)
    
df = pd.concat(dataframes, ignore_index=True)

In [8]:
df1 = pd.read_parquet("drug_data1.parquet",engine='pyarrow')

In [6]:
df2 = pd.read_parquet("drug_data2.parquet",engine='pyarrow')

In [9]:
df = pd.concat([df,df1,df2])
del df1
del df2

In [10]:
df.columns = df.columns.str.lower()

df = df.groupby(df.columns, axis=1).first()

In [11]:
df

,date_time_scraped,name,raw_html,source_name,source_url
0,16/05/2023 13:52:52,Acne keloidalis nuchae,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/a-z/acne-k...
1,16/05/2023 13:54:43,Acne scars,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/acne/derm-...
2,16/05/2023 13:53:30,Actinic keratosis﻿,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/skin-cance...
3,16/05/2023 13:59:10,Alopecia areata﻿,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/hair-loss/...
4,16/05/2023 13:54:07,Athlete's foot﻿,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/a-z/athlet...
...,...,...,...,...,...
39995,28/10/2022 00:50:25,zylofuramine,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",wikipedia.com,https://en.wikipedia.org//wiki/Zylofuramine
39996,28/10/2022 00:50:26,Zyloprim,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",wikipedia.com,https://en.wikipedia.org//wiki/Zyloprim
39997,28/10/2022 00:50:26,Zymar,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",wikipedia.com,https://en.wikipedia.org//wiki/Zymar
39998,28/10/2022 00:50:27,Zyprexa,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",wikipedia.com,https://en.wikipedia.org//wiki/Zyprexa


In [12]:
df['concept_name'] = df['name']

# Drop 'disease_name' and 'name' columns
df.drop(['name'], axis=1, inplace=True)

In [ ]:
df['concept_name'] = df['disease_name'].fillna('') + ' ' + df['name'].fillna('')

# Drop 'disease_name' and 'name' columns
df.drop(['disease_name', 'name'], axis=1, inplace=True)

In [13]:
path = "C:\\Users\\gwenk\\OneDrive\\Documents\\Research\\Scraping\\MetaMap\\public_mm\\"
df_cui = pd.concat([ pd.read_parquet(path+"Drug_CUI_Mappings.parquet",engine='pyarrow'),
                    pd.read_parquet(path+"Disease_CUI_Mappings.parquet",engine='pyarrow')
                   ])

In [14]:
df_cui.reset_index(inplace=True)
df_cui

,index,string_id,cui
0,0,Esoterica Sensitive Skin,
1,1,femstat,
2,2,doliracetam,
3,3,Ciprofloxacin Extended Release Tablets,
4,4,Avar-E Emollient,
...,...,...,...
82827,27435,kleefstra syndrome 2,C4540395
82828,27436,coronavirus infectious disease,"C0009450, C0206750, C0206419, C3714515"
82829,27437,3-methylcrotonyl-coa carboxylase 2 deficiency,C1859499
82830,27438,hyperekplexia 4,C4693933


In [15]:
df_cui[df_cui["cui"] == '']

,index,string_id,cui
0,0,Esoterica Sensitive Skin,
1,1,femstat,
2,2,doliracetam,
3,3,Ciprofloxacin Extended Release Tablets,
4,4,Avar-E Emollient,
...,...,...,...
80929,25537,dioctophymiasis,
81431,26039,multisystem proteinopathy,
81933,26541,pericytoma with t,
82338,26946,tubulinopathy,


In [16]:
cui_dict = {}
for row in df_cui.itertuples():
    cui_dict[row[2]] = row[3]

In [17]:
import re

In [18]:
def find_cui(string):
    string = string.strip()
    string = string.lower()
    string = re.sub(r'/', ' ', string)
    string = re.sub(r':.*', '', string)
    string = re.sub(r'—.*', '', string)
    string = re.sub(r'\([^)]*\)', '', string)
    string = re.sub(r'[^,\-a-zA-Z0-9 ]', '', string)
    try:
        return cui_dict[string.strip()]
    except:
        return ""

In [19]:
df["cuis"] = df["concept_name"].apply(lambda x: find_cui(x))

In [20]:
df_cui.loc[df_cui["string_id"].str.contains("c difficile")]

,index,string_id,cui
55526,134,c difficile,
65370,9978,c difficile infection,"C0009450, C3714514"
77712,22320,c difficile,


In [21]:
df

,date_time_scraped,raw_html,source_name,source_url,concept_name,cuis
0,16/05/2023 13:52:52,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/a-z/acne-k...,Acne keloidalis nuchae,"C0702166, C0001144, C0406631"
1,16/05/2023 13:54:43,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/acne/derm-...,Acne scars,"C0702166, C0001144, C2004491, C0241158"
2,16/05/2023 13:53:30,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/skin-cance...,Actinic keratosis﻿,C0022602
3,16/05/2023 13:59:10,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/hair-loss/...,Alopecia areata﻿,"C0002170, C0002171"
4,16/05/2023 13:54:07,"<html lang=""en""><head>\n <meta charset=""utf...",AADA,https://www.aad.org/public/diseases/a-z/athlet...,Athlete's foot﻿,C0040259
...,...,...,...,...,...,...
39995,28/10/2022 00:50:25,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",wikipedia.com,https://en.wikipedia.org//wiki/Zylofuramine,zylofuramine,C2826074
39996,28/10/2022 00:50:26,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",wikipedia.com,https://en.wikipedia.org//wiki/Zyloprim,Zyloprim,
39997,28/10/2022 00:50:26,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",wikipedia.com,https://en.wikipedia.org//wiki/Zymar,Zymar,
39998,28/10/2022 00:50:27,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",wikipedia.com,https://en.wikipedia.org//wiki/Zyprexa,Zyprexa,


In [22]:
# Define the desired column order
desired_order = [
    'concept_name', 'cuis', 'source_name', 'source_url',
    'date_time_scraped', 'raw_html'
]

# Rearrange the columns in the desired order
df = df[desired_order]

In [23]:
df = df.drop_duplicates(subset=["source_url"]).reset_index()

In [25]:
df = df.drop(df.columns[[0]],axis=1)

In [26]:
df

,concept_name,cuis,source_name,source_url,date_time_scraped,raw_html
0,Acne keloidalis nuchae,"C0702166, C0001144, C0406631",AADA,https://www.aad.org/public/diseases/a-z/acne-k...,16/05/2023 13:52:52,"<html lang=""en""><head>\n <meta charset=""utf..."
1,Acne scars,"C0702166, C0001144, C2004491, C0241158",AADA,https://www.aad.org/public/diseases/acne/derm-...,16/05/2023 13:54:43,"<html lang=""en""><head>\n <meta charset=""utf..."
2,Actinic keratosis﻿,C0022602,AADA,https://www.aad.org/public/diseases/skin-cance...,16/05/2023 13:53:30,"<html lang=""en""><head>\n <meta charset=""utf..."
3,Alopecia areata﻿,"C0002170, C0002171",AADA,https://www.aad.org/public/diseases/hair-loss/...,16/05/2023 13:59:10,"<html lang=""en""><head>\n <meta charset=""utf..."
4,Athlete's foot﻿,C0040259,AADA,https://www.aad.org/public/diseases/a-z/athlet...,16/05/2023 13:54:07,"<html lang=""en""><head>\n <meta charset=""utf..."
...,...,...,...,...,...,...
81195,zylofuramine,C2826074,wikipedia.com,https://en.wikipedia.org//wiki/Zylofuramine,28/10/2022 00:50:25,"<!DOCTYPE html>\n<html class=""client-nojs"" dir..."
81196,Zyloprim,,wikipedia.com,https://en.wikipedia.org//wiki/Zyloprim,28/10/2022 00:50:26,"<!DOCTYPE html>\n<html class=""client-nojs"" dir..."
81197,Zymar,,wikipedia.com,https://en.wikipedia.org//wiki/Zymar,28/10/2022 00:50:26,"<!DOCTYPE html>\n<html class=""client-nojs"" dir..."
81198,Zyprexa,,wikipedia.com,https://en.wikipedia.org//wiki/Zyprexa,28/10/2022 00:50:27,"<!DOCTYPE html>\n<html class=""client-nojs"" dir..."


In [ ]:
first_column = df[df['cui'].isnull()]

In [ ]:
first_column = first_column.drop(first_column.columns[[2,3,4,5,6,7]],axis=1)

In [ ]:
outputfile = "drug_names.txt"
names = set(df["concept_name"])
with open(outputfile,'w',encoding='utf-8') as o:
    for name in names:
        o.write(f"{name}\n")

In [ ]:


# Save the first column as a text file
file_path = 'CUIMatching\\disease_names.txt'
with open(file_path, 'w',encoding='utf-8') as file:
    for index, row in first_column.iterrows():
        file.write(f"{row[0].lower().strip()}\n")

In [ ]:
file_paths = ["CUIMatching\\matched_cuis.csv","CUIMatching\\matched_cuis1.csv","CUIMatching\\matched_cuis2.csv","CUIMatching\\matched_cuis3.csv"]
dfs = []
for path in file_paths:
    temp = pd.read_csv(path,sep="{S}",engine='python')
    dfs.append(temp)
matched_cuis_df = pd.concat(dfs,ignore_index=True)
matched_cuis_df

In [ ]:
import re

In [ ]:
cui_match_dict = dict()
for index,row in matched_cuis_df.iterrows():
    string = row[1]
    string = re.sub(r':.*', '', string).strip().lower()
    string = re.sub(r'—.*', '', string)
    string = re.sub(r'\([^)]*\)', '', string)
    string = re.sub(r'[^,\-a-zA-Z0-9 ]', '', string)
    cui_match_dict[string] = row[0]

In [ ]:
cui_match_dict

In [ ]:
tempdf

In [ ]:
tempdf = df
for index, row in tempdf.iterrows():
    if row['cui'] is not None:
        continue
    try:
        string = row['concept_name']
        string = re.sub(r':.*', '', string).strip().lower()
        string = re.sub(r'—.*', '', string)
        string = re.sub(r'\([^)]*\)', '', string)
        string = re.sub(r'[^,\-a-zA-Z0-9 ]', '', string).strip()
        tempdf.iloc[index]['cui'] = cui_match_dict[string]
    except KeyError:
        print("KeyError at key: ",row[0]," and index: ",index)
    except:
        print("Other error at index: ",index)

In [ ]:
df = tempdf

In [27]:
pd.set_option('display.max_rows', 500)

In [28]:
pd.set_option('display.max_colwidth',500)

In [ ]:
tempdf

In [ ]:
tempdf.to_parquet("CUIMatching\\CUIMatched_Concepts.parquet",engine='pyarrow',compression='snappy')

In [29]:
import re

In [30]:
def remove_duplicate_whitespace(x):
    return re.sub(r"(\. )+",". ",re.sub(r"\.+",".",re.sub(r" +|\r+|\t+"," ",re.sub(r"\n+","\n",x))))

In [33]:
def source_differentiation(source_name,raw_html,concept):
    
    try:
        soup = BeautifulSoup(raw_html,'lxml')

        if source_name == 'Healthy Western Australians (gov)':
            # Code for 'Healthy Western Australians (gov)'
            if "CAHS - 404" in raw_html:
                return ""
            text = ""
            text = text + remove_duplicate_whitespace(soup.find('div',class_="clearfix articlePage content-wrap").text)
            substrings = [
                "This publication is provided for education and information purposes only.",
                "View and download this information as a PDF fact sheet ",
                "Acknowledgements ",
                "Last reviewed: "
            ]
            for substring in substrings:
                text = text.split(substring)[0]


            pass
        elif source_name == 'Illinois DPH':

            text = ""
    #         try:
            tts = soup.find_all('div',class_="cmp-text")
            for t in tts:
                text += t.text
            text = remove_duplicate_whitespace(text)
            # Code for 'Illinois DPH'
            pass
        elif source_name == 'NIH':
            # Code for 'NIH'
            pass
        elif source_name == 'AADA':
            # col-lg-9
            try:
                text = ""
                for p in soup.find('main',class_="col-lg-9").find_all('p'):
                    text += p.text + ". "
                text = remove_duplicate_whitespace(text)
            except:
                pass
            # Code for 'AADA'
            pass
        elif source_name == 'Mayoclinic':
            text = ""
    #         try:
            # text = text + remove_duplicate_whitespace(soup.find('div',class_="content").text)
    #         except:
    #             pass
            # Code for 'Mayoclinic'
            pass
        elif source_name == 'MedlinePlus':
            text = ""
            text += soup.find('div',id="topic-summary").text
            text = remove_duplicate_whitespace(text)
            # Code for 'MedlinePlus'
            pass
        elif source_name == 'NIH NINDS':
            text = ""
            text += soup.find('div',class_="clearfix text-formatted field field--name-body field--type-text-with-summary field--label-hidden field__item").text
            text = remove_duplicate_whitespace(text)
            # Code for 'NIH NINDS'
            pass
        elif source_name == 'CanadaPublicHealth':

            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="mwsbodytext text parbase section").text)
            except:
                pass
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="mwsgeneric-base-html parbase section").text)
            except:
                pass
            # Code for 'CanadaPublicHealth'
            pass
        elif source_name == 'HealthDirectGovAU':
    #         cont_layout-standout (for overview)
            text = ""

            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="main_content-body-text").text)
            except:
                pass

            # Code for 'HealthDirectGovAU'
            pass
        elif source_name == 'Hexahealth':
            # Code for 'Hexahealth'
            pass
        elif source_name == 'NHSScot':
            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="editor").text)
            except:
                pass
            # Code for 'NHSScot'
            pass
        elif source_name == 'Cleveland Clinic':
            text = remove_duplicate_whitespace(soup.find('div',class_="js-health-article__content health-article__content").text)
            # Code for 'Cleveland Clinic'
    #         js-section js-section--overview
            pass
        elif source_name == 'DrugsDOTcom':
            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="ddc-expand-read-more").text)
            except:
                pass
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="ddc-expand-read-more ddc-expand-read-more-hidden").text)
            except:
                pass
            try:
                for tr in soup.find('div',class_="ddc-clear-both injection-ignore-inside").find_all('tr')[1:]:
                    text += tr.text
            except: 
                pass
            # Code for 'DrugsDOTcom'
            pass
        elif source_name == 'CDC':
            # Code for 'CDC'
            text = ""
            pass
        elif source_name == 'Rare Diseases NIH':
            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="fs-md-18").text)
            except:
                pass

            # Code for 'Rare Diseases NIH'
            pass
        elif source_name == 'Medscape':
            text = soup.text
            # Code for 'Medscape'
            pass
        elif source_name == 'NHS':

            text = ""
    #         try:
            sections = soup.find_all("section")
            for section in sections[1:]:
                text = text + remove_duplicate_whitespace(section.text)
    #         except:
    #             pass
            # Code for 'NHS'
            pass
        elif source_name == 'Clinical Trials gov':
            text = ""
            # Code for 'Clinical Trials gov'
            pass
        elif source_name == 'Rheumatology':
            # Code for 'Rheumatology'
            pass
        elif source_name == 'Merck Manuals':
            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="topic__accordion").text)
            except:
                pass
            # Code for 'Merck Manuals'
            pass
        elif source_name == 'Wikipedia':

            # Code for 'Wikipedia'
            pass
        elif source_name == 'Wisconsin Department of Health Services':

            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="c-field__content").text)
            except:
                pass
            # Code for 'Wisconsin Department of Health Services'
            pass
        elif source_name == 'WHO':
            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="sf_colsOut tabContent").text)
            except:
                pass
            # Code for 'WHO'
            pass
        elif source_name == 'Seattle Childrens':

            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="main-content-body").text)
            except:
                pass
            # Code for 'Seattle Childrens'
            pass
        elif source_name == 'WebMD':

            text = ""
            try:
                text = text + remove_duplicate_whitespace(soup.find('div',class_="article-page active-page").text)
            except:
                pass
            # Code for 'WebMD'
            pass
        elif source_name == 'RareDiseases.org':
            try:
                text = soup.find('div',id="disease-overview").text.strip()
            except:
                text = None
            # Code for 'RareDiseases.org'
            pass
        else:
            crash = 0/0/0/0
        return text
        print(remove_duplicate_whitespace(text))
    except:
        print(concept)
        return ""

In [43]:
df[df["concept_name"].str.contains("Eosinophilic Esophagitis")]

,concept_name,cuis,source_name,source_url,date_time_scraped,raw_html
2247,Eosinophilic Esophagitis,C0341106,Cleveland Clinic,https://my.clevelandclinic.org/health/diseases/14321-eosinophilic-esophagitis,06/06/2023 19:26:46,"<html dir=""ltr"" lang=""en""><head>\n<meta charset=""utf-8"">\n<meta name=""viewport"" content=""width=device-width, initial-scale=1.0"">\n<link rel=""preconnect"" href=""https://fonts.googleapis.com"">\n<link rel=""preconnect"" href=""https://fonts.gstatic.com"" crossorigin="""">\n<link href=""//fonts.googleapis.com/css2?family=Roboto+Condensed&amp;family=Roboto:wght@400;500;700;900&amp;display=swap"" rel=""stylesheet"" as=""style"" onload=""this.rel='stylesheet'"">\n<link rel=""preload"" href=""https://cdn.static.zdbb...."
6977,EE (Eosinophilic Esophagitis),,DrugsDOTcom,https://www.drugs.com/condition/eosinophilic-esophagitis.html,11/05/2023 19:43:43,"<html lang=""en""><head>\n<meta charset=""utf-8"">\n<title>List of 8 Eosinophilic Esophagitis Medications Compared - Drugs.com</title>\n<meta name=""viewport"" content=""width=device-width, initial-scale=1"">\n<meta name=""description"" content=""Compare risks and benefits of common medications used for Eosinophilic Esophagitis. Find the most popular drugs, view ratings and user reviews."">\n<meta name=""referrer"" content=""origin-when-cross-origin"">\n<meta name=""robots"" content=""max-image-preview:large"">..."
10561,Eosinophilic Esophagitis,C0341106,MedlinePlus,https://medlineplus.gov/eosinophilicesophagitis.html,11/05/2023 17:06:04,"<html id=""health_topic"" class=""us"" data-root=""https://medlineplus.gov/"" lang=""en""><head>\n\n <meta charset=""utf-8"">\n <meta http-equiv=""X-UA-Compatible"" content=""IE=edge,chrome=1"">\n <meta http-equiv=""window-target"" content=""_top"">\n <meta http-equiv=""Content-Type"" content=""text/html; charset=UTF-8"">\n <meta name=""viewport"" content=""width=device-width, initial-scale=1"">\n\n \n\n \n\n <link rel=""canonical"" href=""https://medlineplus.gov/e..."
13345,Eosinophilic Esophagitis,C0341106,Merck Manuals,https://www.merckmanuals.com/professional/gastrointestinal-disorders/esophageal-and-swallowing-disorders/eosinophilic-esophagitis,12/05/2023 18:17:33,"<html dir=""ltr"" style="""" class="" no-touchevents flexbox flexwrap csscolumns csscolumns-width csscolumns-span csscolumns-fill csscolumns-gap csscolumns-rule csscolumns-rulecolor csscolumns-rulestyle csscolumns-rulewidth no-csscolumns-breakbefore no-csscolumns-breakafter no-csscolumns-breakinside"" lang=""en-US""><head>\n \n<link rel=""preconnect"" href=""https://cdn.cookielaw.org"">\n<link rel=""preconnect"" href=""https://www.googletagmanager.com"">\n<link href=""https://cdn.cookielaw.org/scripttempl..."


In [ ]:
test_index = 8877
print(df.iloc[test_index])
source_differentiation(df.iloc[test_index].source_name,df.iloc[test_index].raw_html)

In [35]:
from tqdm import tqdm

In [36]:
tqdm.pandas()

In [37]:
df["clean_text"] = df.progress_apply(lambda x: source_differentiation(x.source_name,x.raw_html,x.concept_name),axis=1)

 11%|████████▌                                                                    | 9035/81200 [04:09<14:37, 82.21it/s]

Abdominal Pain
Abscess in Breast
ACL Tear


 11%|████████▌                                                                    | 9044/81200 [04:10<24:53, 48.32it/s]

Acute Respiratory Infection in Children
Acute Liver Failure
Adenoids
Alcoholic Liver Disease
Allergic Rhinitis
Amblyopia
Anal cancer
Anal Fissure


 11%|████████▌                                                                    | 9051/81200 [04:10<37:48, 31.81it/s]

...

 13%|█████████▉                                                                  | 10568/81200 [05:56<35:31, 33.14it/s]

Eosinophilic Esophagitis
Esophageal Cancer


 13%|█████████▉                                                                  | 10577/81200 [05:56<39:38, 29.69it/s]

Exercise and Physical Fitness
Eye Infections




KeyboardInterrupt

